In [171]:
import os
import spacy
import numpy as np
#
import pandas as pd
#
from spacy.matcher import Matcher

In [163]:
nlp = spacy.load('de_core_news_md')
pd.set_option('display.max_colwidth', None)
os.environ["SPACY_WARNING_IGNORE"] = "W008"

In [164]:
REVIEWS_CLEANED_FILE_PATH = 'data/reviews-clean.csv'

df = pd.read_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')
df.head()

,Unnamed: 0,caption,rating,timestamp
0,0,abstand leckerst pizza halb abends mitnehmen bestellen positiv emotion einfach niederknien,5.0,2023-02-15 22:10:07.146586
1,4,chicken pizzas,4.0,2023-02-14 00:10:07.146703
2,7,leck pizz cool atmosphäre,5.0,2023-02-13 00:10:07.146710
3,11,extrem lecker empfehlung,5.0,2023-02-10 00:11:07.737920
4,13,gut pizza authentisch,5.0,2023-02-09 00:11:07.737926


In [165]:
categories = ['essen', 'gaststättengewerbe', 'atmosphäre', 'preis', 'warten']

In [166]:
def spacy_most_similar(word, topn=10):
    ms = nlp.vocab.vectors.most_similar(
        nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [167]:
spacy_most_similar(categories[0])

(['Aufessen',
  'weggefuttert',
  'drinken',
  'einzukochen',
  'koten',
  'inhaliere',
  'knabbere',
  'Bio-Essen',
  'schmatz',
  'Trinket'],
 array([[1.    , 0.8243, 0.8131, 0.7734, 0.7115, 0.7059, 0.7032, 0.6845,
         0.672 , 0.6681]], dtype=float32))

In [192]:
matcher = Matcher(nlp.vocab)
patterns = [[{'POS':'ADJ'}, {'POS':'NOUN'}]]
matcher.add("category", patterns)

matches = matcher(nlp('ein gut Fensert'))
print(matches)

'''
for rowIndex, wordText in enumerate(df['caption'][0:2]):
    print(wordText)
    doc = nlp(wordText)
    matches = matcher(doc)
    print(matches)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        print(span.text)
'''

[]


"\nfor rowIndex, wordText in enumerate(df['caption'][0:2]):\n    print(wordText)\n    doc = nlp(wordText)\n    matches = matcher(doc)\n    print(matches)\n    for match_id, start, end in matches:\n        string_id = nlp.vocab.strings[match_id]\n        span = doc[start:end]\n        print(span.text)\n"

In [168]:
similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption'][0:3]):
    word = nlp(wordText)
    tokens = [token.text for token in word]
    #tokens
    allowed_postags = ["NOUN"]
    for i, token in enumerate(word):
        if token.pos_ in allowed_postags:
            if token and token.vector_norm > 1:
                similarity_cat0 = nlp(categories[0]).similarity(token)
                if similarity_cat0 > similarity_min: # fits into
                    print(rowIndex, categories[0], token, similarity_cat0)

                similarity_cat1 = nlp(categories[1]).similarity(token)
                if similarity_cat1 > similarity_min: # fits into
                    print(rowIndex, categories[1], token, similarity_cat1)

                similarity_cat2 = nlp(categories[2]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[2], token, similarity_cat2)

                similarity_cat3 = nlp(categories[3]).similarity(token)
                if similarity_cat3 > similarity_min: # fits into
                    print(rowIndex, categories[3], token, similarity_cat3)

                similarity_cat4 = nlp(categories[4]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[4], token, similarity_cat4)


0 essen pizza 0.43865881948792484
0 essen bestellen 0.4143392967224167
0 preis bestellen 0.4073594233690193
0 atmosphäre emotion 0.4534038094758903
0 warten emotion 0.02380392906254289
2 atmosphäre atmosphäre 1.0
2 warten atmosphäre 0.15941722217303403


/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/3461502951.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(token)


In [169]:
nlp("essen").similarity(nlp("pizza"))

0.4386588382204547

In [170]:
nlp('pizz').vector_norm

0